# Project Deep Dive: NPC Lore LoRA Training & Merging

This notebook automates the process of fine-tuning, merging, and quantizing a Llama 3 8B model with custom lore for the Project Deep Dive game.

### Workflow:
1.  **Configuration:** Set your desired output name and training parameters in Cell 2.
2.  **Login:** Run Cell 4 to log into Hugging Face (only needs to be done once).
3.  **Training:** Run Cell 6 to train the LoRA adapter using your GPU.
4.  **Merge & Quantize:** Run Cells 8, 9, and 10 to merge the LoRA into the base model and create a final GGUF file.
5.  **Deployment:** Load your new, custom `...-merged.gguf` file directly into LM Studio.

In [7]:
import os
import json
import subprocess

# --- 1. CORE CONFIGURATION ---
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
LORA_OUTPUT_NAME = "ProjectDeepDive-Lora-v1"
DATASET_NAME = "lore_training_data"

# --- 2. TRAINING HYPERPARAMETERS ---
EPOCHS = 5.0
BATCH_SIZE = 1
GRADIENT_ACCUMULATION = 4

# --- 3. SCRIPT SETUP (No need to edit below this line) ---
PROJECT_ROOT = os.path.abspath("..")
model_folder_name = MODEL_ID.split('/')[-1]
output_dir_relative = os.path.join("saves", model_folder_name, LORA_OUTPUT_NAME)
training_script_path_relative = os.path.join("src", "train.py")
dataset_file_path_local = f"{DATASET_NAME}.json"

if not os.path.exists(dataset_file_path_local):
    raise FileNotFoundError(f"CRITICAL: Dataset file not found at '{os.path.abspath(dataset_file_path_local)}'. Make sure '{dataset_file_path_local}' is in the same folder as this notebook.")

print("✅ Configuration loaded successfully.")
print(f"   Model ID: {MODEL_ID}")
print(f"   Dataset: {DATASET_NAME}")
print(f"   Project Root: {PROJECT_ROOT}")
print(f"   Output will be saved to: {os.path.join(PROJECT_ROOT, output_dir_relative)}")

✅ Configuration loaded successfully.
   Model ID: meta-llama/Meta-Llama-3-8B-Instruct
   Dataset: lore_training_data
   Project Root: c:\Users\ruben\Documents\TrainingAI\LLaMA-Factory
   Output will be saved to: c:\Users\ruben\Documents\TrainingAI\LLaMA-Factory\saves\Meta-Llama-3-8B-Instruct\ProjectDeepDive-Lora-v1


In [8]:
# Verify the dataset can be loaded and count the entries
try:
    with open(dataset_file_path_local, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    num_instructions = len(data) # Changed this to reflect the new structure
    print(f"✅ Dataset '{dataset_file_path_local}' loaded successfully.")
    print(f"   Found {num_instructions} question/answer pairs for training.")
    if num_instructions < 10:
        print("   ⚠️ WARNING: Dataset is very small. Consider adding more examples for better results.")
except Exception as e:
    print(f"❌ ERROR: Failed to read or parse the dataset file. Please check for syntax errors in your JSON.")
    print(f"   Details: {e}")

✅ Dataset 'lore_training_data.json' loaded successfully.
   Found 7 question/answer pairs for training.
   ⚠️ WARNING: Dataset is very small. Consider adding more examples for better results.


In [ ]:
import subprocess
# Replace with your NEW token
hf_token = ""
login_command = ["huggingface-cli", "login", "--token", hf_token]
result = subprocess.run(login_command, capture_output=True, text=True)
if result.returncode == 0:
    print("✅ Successfully logged in to Hugging Face.")
else:
    print("❌ Failed to log in to Hugging Face. Check your token and network connection.")
    print(result.stdout); print(result.stderr)

Exception in thread Thread-5 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "c:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 10: character maps to <undefined>


✅ Successfully logged in to Hugging Face.


In [10]:
command = [
    "python", training_script_path_relative,
    "--model_name_or_path", MODEL_ID,
    "--do_train",
    "--dataset", DATASET_NAME,
    "--finetuning_type", "lora",
    "--output_dir", output_dir_relative,
    "--lora_target", "all",
    "--per_device_train_batch_size", str(BATCH_SIZE),
    "--gradient_accumulation_steps", str(GRADIENT_ACCUMULATION),
    "--num_train_epochs", str(EPOCHS),
    "--plot_loss",
    "--fp16"
]
print("--- Training Command ---")
print(subprocess.list2cmdline(command))
print("------------------------")

--- Training Command ---
python src\train.py --model_name_or_path meta-llama/Meta-Llama-3-8B-Instruct --do_train --dataset lore_training_data --finetuning_type lora --output_dir saves\Meta-Llama-3-8B-Instruct\ProjectDeepDive-Lora-v1 --lora_target all --per_device_train_batch_size 1 --gradient_accumulation_steps 4 --num_train_epochs 5.0 --plot_loss --fp16
------------------------


In [11]:
print("🚀 Starting training... This may take a while.")
process = subprocess.Popen(command, cwd=PROJECT_ROOT, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', bufsize=1)
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None: break
    if output: print(output.strip())
if process.returncode == 0:
    print("\n🎉 Training finished successfully! 🎉")
else:
    print(f"\n❌ Training failed with exit code {process.returncode}.")

🚀 Starting training... This may take a while.
Traceback (most recent call last):
File "c:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\import_utils.py", line 2317, in __getattr__
module = self._get_module(self._class_to_module[name])
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
File "c:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\import_utils.py", line 2347, in _get_module
raise e
File "c:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\import_utils.py", line 2345, in _get_module
return importlib.import_module("." + module_name, self.__name__)
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
File "c:\Users\ruben\AppData\Local\Programs\Python\Python311\Lib\importlib\__init__.py", line 126, in import_module
return _bootstrap._gcd_import(name[level:], package, level)
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
File "<frozen importlib._bootstr

### Step 2: Merge LoRA and Quantize to GGUF

Now that the LoRA adapter is trained, we will perform two final steps:
1.  **Merge:** Combine the base Llama 3 model with our LoRA adapter to create a new, full-sized (unquantized) model.
2.  **Quantize:** Compress the large, merged model into a single, efficient GGUF file that LM Studio can use.

In [12]:
# --- MERGE THE TRAINED LORA ---
print("🚀 Starting model merge process...")

# Define the directory where the full-precision merged model will be saved
MERGED_MODEL_DIR_RELATIVE = os.path.join("merged_models", f"{model_folder_name}-{LORA_OUTPUT_NAME}")
MERGED_MODEL_DIR_ABSOLUTE = os.path.join(PROJECT_ROOT, MERGED_MODEL_DIR_RELATIVE)

merge_command = [
    "python", os.path.join("src", "export_model.py"),
    "--model_name_or_path", MODEL_ID,
    "--adapter_name_or_path", output_dir_relative,
    "--template", "llama3",
    "--export_dir", MERGED_MODEL_DIR_RELATIVE,
    "--export_size", "2" # Shard the model into 2GB chunks
]

print("--- Merge Command ---")
print(subprocess.list2cmdline(merge_command))
print("---------------------")

process = subprocess.Popen(merge_command, cwd=PROJECT_ROOT, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', bufsize=1)
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None: break
    if output: print(output.strip())

if process.returncode == 0:
    print(f"\n🎉 Model merged successfully! Full-precision model saved at:\n{MERGED_MODEL_DIR_ABSOLUTE}")
else:
    print(f"\n❌ Model merge failed with exit code {process.returncode}.")

🚀 Starting model merge process...
--- Merge Command ---
python src\export_model.py --model_name_or_path meta-llama/Meta-Llama-3-8B-Instruct --adapter_name_or_path saves\Meta-Llama-3-8B-Instruct\ProjectDeepDive-Lora-v1 --template llama3 --export_dir merged_models\Meta-Llama-3-8B-Instruct-ProjectDeepDive-Lora-v1 --export_size 2
---------------------
python: can't open file 'c:\\Users\\ruben\\Documents\\TrainingAI\\LLaMA-Factory\\src\\export_model.py': [Errno 2] No such file or directory

❌ Model merge failed with exit code 2.


In [13]:
# --- QUANTIZE THE MERGED MODEL TO GGUF ---
print("\n🚀 Starting quantization to GGUF format...")

# Path to the llama.cpp repository (should be next to LLaMA-Factory)
LLAMA_CPP_DIR = os.path.abspath(os.path.join(PROJECT_ROOT, "..", "llama.cpp"))

if not os.path.isdir(LLAMA_CPP_DIR):
    raise NotADirectoryError(f"CRITICAL: llama.cpp directory not found at '{LLAMA_CPP_DIR}'. Please ensure it's cloned in the same folder as LLaMA-Factory.")

# Define the final output file for our game
FINAL_GGUF_DIR = os.path.join(PROJECT_ROOT, "final_gguf_models")
os.makedirs(FINAL_GGUF_DIR, exist_ok=True)
FINAL_GGUF_FILE = os.path.join(FINAL_GGUF_DIR, f"{model_folder_name}-{LORA_OUTPUT_NAME}-Q4_K_M.gguf")

quantize_command = [
    "python", os.path.join(LLAMA_CPP_DIR, "convert-hf-to-gguf.py"),
    MERGED_MODEL_DIR_ABSOLUTE,
    "--outfile", FINAL_GGUF_FILE,
    "--outtype", "q4_k_m" # A high-quality, medium-sized quantization type
]

print("--- Quantize Command ---")
print(subprocess.list2cmdline(quantize_command))
print("------------------------")

process = subprocess.Popen(quantize_command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', bufsize=1)
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None: break
    if output: print(output.strip())

if process.returncode == 0:
    print(f"\n🎉 Quantization successful! Your game-ready model is located at:\n{FINAL_GGUF_FILE}")
    print("\n💡 You can now delete the large merged model folder to save space:")
    print(f"   {MERGED_MODEL_DIR_ABSOLUTE}")
else:
    print(f"\n❌ Quantization failed with exit code {process.returncode}.")


🚀 Starting quantization to GGUF format...
--- Quantize Command ---
python c:\Users\ruben\Documents\TrainingAI\llama.cpp\convert-hf-to-gguf.py c:\Users\ruben\Documents\TrainingAI\LLaMA-Factory\merged_models\Meta-Llama-3-8B-Instruct-ProjectDeepDive-Lora-v1 --outfile c:\Users\ruben\Documents\TrainingAI\LLaMA-Factory\final_gguf_models\Meta-Llama-3-8B-Instruct-ProjectDeepDive-Lora-v1-Q4_K_M.gguf --outtype q4_k_m
------------------------
python: can't open file 'c:\\Users\\ruben\\Documents\\TrainingAI\\llama.cpp\\convert-hf-to-gguf.py': [Errno 2] No such file or directory

❌ Quantization failed with exit code 2.


### Workflow Complete!

1.  **Locate Your Final Model:**
    *   Navigate to your `LLaMA-Factory` folder.
    *   You will find a new folder named `final_gguf_models`.
    *   Inside is your custom, ready-to-use GGUF file (e.g., `Meta-Llama-3-8B-Instruct-ProjectDeepDive-Lora-v1-Q4_K_M.gguf`).

2.  **Load in LM Studio:**
    *   Open LM Studio.
    *   You can either move this new GGUF file to your `.cache/lm-studio/models` folder, or simply drag-and-drop it from the `final_gguf_models` folder directly onto the LM Studio window.
    *   It will now appear in your "My Models" list.

3.  **Activate and Test:**
    *   Select your new, custom model from the dropdown at the top of the Chat or Server tab.
    *   Start the local server. **You do NOT need to load any LoRA adapters separately.**
    *   Launch your Unity game and interact with an NPC to test the new, lore-aware responses!